### Rival posession overall

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#df = pd.read_csv('/Users/pgule/Documents/Sample Data/epl.csv', delimiter=',')
df = pd.read_csv('/Users/luisalawson/Downloads/Sample Data/epl.csv', delimiter=';')

In [26]:
def group_plays(df):
    match_stats_list = []
    goalkeeper_events = ['Attempt saved', 'Save', 'Pass', 'Keeper pick-up']
    unique_matches = df['match_id'].unique()

    for currentmatch in unique_matches:
        match_df = df[df['match_id'] == currentmatch]

        home_team_id = match_df['home_team_id'].iloc[0]
        away_team_id = match_df['away_team_id'].iloc[0]
        home_team_name = match_df['home_team_name'].iloc[0]
        away_team_name = match_df['away_team_name'].iloc[0]

        home_plays = 0
        away_plays = 0
        home_passes = 0
        away_passes = 0

        current_team = None
        last_event_id = None
        in_play = False
        current_passes = 0
        shot_on_goal = False
        home_team_first_half_side = None
        away_team_first_half_side = None
        home_team_danger = None
        away_team_danger = None

        for i, row in match_df.iterrows():
            team_id = row['team_id']
            event_id = row['event_id']

            if not in_play:
                in_play = True
                current_team = team_id
                last_event_id = event_id
                current_passes = 0
            else:
                if team_id != current_team or event_id <= last_event_id:
                    # Play ends, record passes
                    if current_team == home_team_id:
                        home_plays += 1
                        home_passes += current_passes
                    else:
                        away_plays += 1
                        away_passes += current_passes
                    # New play starts
                    current_team = team_id
                    last_event_id = event_id
                    current_passes = 0
                else:
                    # Continue current play
                    last_event_id = event_id
            
            # Check for shot on goal
            if (row['description'] == 'Goal') and (row['outcome'] == 1) and x_coordinate > 50:
                shot_on_goal = True

            # Determine the first-half side for the home team
            if home_team_first_half_side is None:
                if (row['playerPosition'] == 'Goalkeeper') and (row['period_id'] == 1) and (row['description'] in goalkeeper_events) and (row['team_id']== home_team_id): 
                    x_coordinate = float(row['x'].replace(',', '.'))
                    if x_coordinate >50:
                        home_team_first_half_side = (50,100)
                        home_team_danger = (70,100)
                    else:
                        home_team_first_half_side = (0,50)
                        home_team_danger = (0,30)

        
            # Count successful passes in the current play
            if row['description'] == 'Pass' and row['outcome'] == 1:
                current_passes += 1
        
        if in_play:
            if current_team == home_team_id:
                home_plays += 1
                home_passes += current_passes
            else:
                away_plays += 1
                away_passes += current_passes
        
        if home_team_first_half_side == (0,50):
            away_team_first_half_side = (50.1,100)
            away_team_danger = (70,100)
        else:
            away_team_first_half_side = (0,50)
            away_team_danger = (0,30)

        match_stats = {
            'match_id': currentmatch,
            'home_team': home_team_name,
            'away_team': away_team_name,
            'home_team_plays': home_plays,
            'away_team_plays': away_plays,
            'home_passes_per_play': home_passes / home_plays if home_plays > 0 else 0,
            'away_passes_per_play': away_passes / away_plays if away_plays > 0 else 0,
            'home_team_first_half_side': home_team_first_half_side,
            'away_team_first_half_side': away_team_first_half_side,
            'home_team_danger':home_team_danger,
            'away_team_danger':away_team_danger
        }

        match_stats_list.append(match_stats)
    
    return pd.DataFrame(match_stats_list)


In [27]:
df_grouped_plays = group_plays(df)

In [28]:
df_dc = pd.read_csv('/Users/luisalawson/Downloads/dc_results.csv')

In [30]:
df_dc['ranking_overall'] = df_dc['dc_overall'].rank(ascending=False)
df_dc['ranking_home'] = df_dc['dc_home'].rank(ascending=False)
df_dc['ranking_away'] = df_dc['dc_away'].rank(ascending=False)

In [31]:
df_dc.head()

,team_name,dc_overall,dc_home,dc_away,dc_home_dif,dc_away_dif,ranking_overall,ranking_home,ranking_away
0,Manchester United,0.716372,0.715204,0.717443,0.001168,-0.001071,5.0,6.0,1.0
1,Southampton,0.718287,0.722656,0.714054,-0.004370,0.004233,3.0,4.0,2.0
2,Liverpool,0.724176,0.739555,0.710407,-0.015379,0.013769,1.0,1.0,3.0
3,Fulham,0.714472,0.719208,0.709649,-0.004736,0.004823,6.0,5.0,4.0
4,Arsenal,0.717665,0.728324,0.707892,-0.010659,0.009773,4.0,3.0,5.0


In [32]:
def get_team_rankings(row, df_dc):
    
    home_team = row['home_team']
    away_team = row['away_team']

    home_team_ranking_overall = df_dc[df_dc['team_name'] == home_team]['ranking_overall'].values
    home_team_ranking_home = df_dc[df_dc['team_name'] == home_team]['ranking_home'].values
    home_team_ranking_away = df_dc[df_dc['team_name'] == home_team]['ranking_away'].values
    home_team_dc_value_home = df_dc[df_dc['team_name'] == home_team]['dc_home'].values
    home_team_dc_value_away= df_dc[df_dc['team_name'] == home_team]['dc_away'].values

    away_team_ranking_overall = df_dc[df_dc['team_name'] == away_team]['ranking_overall'].values
    away_team_ranking_home = df_dc[df_dc['team_name'] == away_team]['ranking_home'].values
    away_team_ranking_away = df_dc[df_dc['team_name'] == away_team]['ranking_away'].values
    away_team_dc_value_home = df_dc[df_dc['team_name'] == away_team]['dc_home'].values
    away_team_dc_value_away= df_dc[df_dc['team_name'] == away_team]['dc_away'].values

    return pd.Series({
        'home_team_ranking_overall': home_team_ranking_overall[0] if len(home_team_ranking_overall) > 0 else None,
        'home_team_ranking_home': home_team_ranking_home[0] if len(home_team_ranking_home) > 0 else None,
        'home_team_ranking_away': home_team_ranking_away[0] if len(home_team_ranking_away) > 0 else None,
        'home_team_dc_value_home': home_team_dc_value_home[0]if len(home_team_dc_value_home) > 0 else None,
        'home_team_dc_value_away': home_team_dc_value_away[0]if len(home_team_dc_value_away) > 0 else None,
        'away_team_ranking_overall': away_team_ranking_overall[0] if len(away_team_ranking_overall) > 0 else None,
        'away_team_ranking_home': away_team_ranking_home[0] if len(away_team_ranking_home) > 0 else None,
        'away_team_ranking_away': away_team_ranking_away[0] if len(away_team_ranking_away) > 0 else None,
        'away_team_dc_value_home': away_team_dc_value_home[0]if len(away_team_dc_value_home) > 0 else None,
        'away_team_dc_value_away': away_team_dc_value_away[0]if len(away_team_dc_value_away) > 0 else None
    })

In [33]:
df_grouped_plays[['home_team_ranking_overall', 'home_team_ranking_home', 'home_team_ranking_away', 'home_team_dc_value_home', 'home_team_dc_value_away', 
                   'away_team_ranking_overall', 'away_team_ranking_home', 'away_team_ranking_away', 'away_team_dc_value_home','away_team_dc_value_away' ]] = df_grouped_plays.apply(get_team_rankings, axis=1, df_dc=df_dc)

In [34]:
condition1 = (df_grouped_plays['home_passes_per_play'] > df_grouped_plays['away_passes_per_play'])
condition2 = (df_grouped_plays['home_team_ranking_overall'] < df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] > df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition1 & condition2, 1, 0)

condition3 = (df_grouped_plays['home_passes_per_play'] < df_grouped_plays['away_passes_per_play'])
condition4 = (df_grouped_plays['home_team_ranking_overall'] > df_grouped_plays['away_team_ranking_overall']) | \
             (df_grouped_plays['home_team_dc_value_home'] < df_grouped_plays['away_team_dc_value_away'])

df_grouped_plays['correct'] = np.where(condition3 & condition4, 1, df_grouped_plays['correct'])

In [35]:
excel_file_path = '/Users/luisalawson/Downloads/pass_dc_comparison.xlsx'  

df_grouped_plays.to_excel(excel_file_path, index=False)
print(f"DataFrame saved to {excel_file_path}")

DataFrame saved to /Users/luisalawson/Downloads/pass_dc_comparison.xlsx


In [36]:
df_grouped_plays[(df_grouped_plays['home_team']=='Manchester United') | (df_grouped_plays['away_team']=='Manchester United')]

,match_id,home_team,away_team,home_team_plays,away_team_plays,home_passes_per_play,away_passes_per_play,home_team_first_half_side,away_team_first_half_side,home_team_danger,...,home_team_ranking_home,home_team_ranking_away,home_team_dc_value_home,home_team_dc_value_away,away_team_ranking_overall,away_team_ranking_home,away_team_ranking_away,away_team_dc_value_home,away_team_dc_value_away,correct
1,442000,Everton,Manchester United,340,338,0.670588,1.789941,"(0, 50)","(50.1, 100)","(0, 30)",...,13.0,14.0,0.694702,0.676357,5.0,6.0,1.0,0.715204,0.717443,1
13,442012,Manchester United,Fulham,294,288,2.044218,1.350694,"(0, 50)","(50.1, 100)","(0, 30)",...,6.0,1.0,0.715204,0.717443,6.0,5.0,4.0,0.719208,0.709649,1
24,442023,Southampton,Manchester United,292,288,1.486301,1.906250,"(0, 50)","(50.1, 100)","(0, 30)",...,4.0,2.0,0.722656,0.714054,5.0,6.0,1.0,0.715204,0.717443,0
34,442033,Manchester United,Wigan Athletic,291,292,1.398625,1.215753,"(0, 50)","(50.1, 100)","(0, 30)",...,6.0,1.0,0.715204,0.717443,7.0,9.0,6.0,0.702730,0.706317,1
41,442040,Liverpool,Manchester United,321,318,1.330218,1.229560,"(0, 50)","(50.1, 100)","(0, 30)",...,1.0,3.0,0.739555,0.710407,5.0,6.0,1.0,0.715204,0.717443,1
54,442053,Manchester United,Tottenham Hotspur,320,317,2.056250,0.662461,"(0, 50)","(50.1, 100)","(0, 30)",...,6.0,1.0,0.715204,0.717443,12.0,11.0,15.0,0.700559,0.673469,1
63,442062,Newcastle United,Manchester United,327,330,1.250765,1.454545,"(0, 50)","(50.1, 100)","(0, 30)",...,17.0,19.0,0.670972,0.646586,5.0,6.0,1.0,0.715204,0.717443,1
72,442071,Manchester United,Stoke City,271,271,2.217712,1.040590,"(0, 50)","(50.1, 100)","(0, 30)",...,6.0,1.0,0.715204,0.717443,8.0,8.0,8.0,0.704946,0.691324,1
82,442081,Chelsea,Manchester United,314,311,1.235669,1.360129,"(0, 50)","(50.1, 100)","(0, 30)",...,2.0,7.0,0.739376,0.698745,5.0,6.0,1.0,0.715204,0.717443,0
92,442091,Manchester United,Arsenal,292,299,1.654110,1.842809,"(0, 50)","(50.1, 100)","(0, 30)",...,6.0,1.0,0.715204,0.717443,4.0,3.0,5.0,0.728324,0.707892,1


### Rival posession in dangerous area